Setup

In [51]:
import benepar
import spacy
import pandas as pd
import decimal
from collections import Counter
import re
from lxml import etree

import apted
from apted import APTED
from apted.helpers import Tree
from itertools import combinations

from math import comb
import time
import json
from datetime import datetime
import os
import numpy as np

benepar.download('benepar_en3')

nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\cedch\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


Helper Methods

In [52]:
# SEXP TO XML
def clean_xml(xml):
    xml = re.sub('<(/?)[^a-zA-Z/][^>]*>', '<\g<1>UNK>', xml) # invalid tokens labeled 'UNK'
    return xml.replace(' ', '')

def sexp_to_xml(sexp):
    def apply_inner_re(s):
        return re.sub('\(([^ ]*) ([^\)\(]*)\)', '<\g<1>> \g<2> </\g<1>>', s)

    xml = apply_inner_re(sexp)
    while xml.startswith('('):
        xml = apply_inner_re(xml)

    with open('special_chars.txt') as f:
        special_chars = dict([line.split() for line in f])

    def key_to_re(s):
        s = re.sub('(.*)([\\\.\+\*\?\^\$\(\)\[\]\{\}\|])(.*)', '\g<1>\\\\\g<2>\g<3>', s)
        return '<(/?)' + s + '>'

    for k, v in special_chars.items():
        xml = re.sub(key_to_re(k), f'<\g<1>{v}>', xml)

    return clean_xml(xml)

# TREE EDIT DISTANCE
def apted_format(parse_str):
    parse_str = re.sub('\(([^ ]+) [^ \(\)]+?\)', '(\g<1>)', parse_str)
    parse_str = parse_str.replace(' ', '')
    parse_str = parse_str.replace('(', '{')
    parse_str = parse_str.replace(')', '}')
    return parse_str

# PARATACTIC CHILDREN STRICT 
def find_parataxis_strict(e):
    global parataxis_clause_tags
    print(e.getchildren())
    children = [c.tag for c in e.getchildren() if not(c.tag.startswith('PUNCT-'))] # excludes punct
    sum = 0
    in_group = False
    print(children)
    
    for i in range(len(children) - 1):
        if children[i] in parataxis_clause_tags and children[i + 1] in parataxis_clause_tags:
            sum += 1
            if not(in_group):
                sum += 1
                in_group = True
        else:
            in_group = False
    return sum if sum != 0 else 1

# LEMMATIZATION
def lemmatize(word: str):
    return lemmatizer(word)[0].lemma_

# AOA
def aoa_of(word: str):
    global aoa_df, aoa_mode
    
    search = aoa_df[aoa_df['Word'] == word]['Rating.Mean']
    if len(search) == 1:
        return float(search)
    
    lemma_search = aoa_df[aoa_df['Word'] == lemmatize(word)]['Rating.Mean']
    if len(lemma_search) == 1:
        return float(lemma_search)
    
    lemmas = aoa_df[aoa_df['Lemma'] == lemmatize(word)]['Rating.Mean']
    if len(lemmas) == 0:
        return -1
    elif len(lemmas) == 1:
        return float(lemmas)
    elif aoa_mode == 'avg':
        return sum(lemmas) / len(lemmas)
    elif aoa_mode == 'max':
        return max(lemmas)
    
    return -1

#Average
def list_avg(l):
    return sum(l) / len(l) if len(l) > 0 else 0


Analysis Functions

In [63]:
clause_tags = ['S', 'SBARQ', 'SINV'] # Not included: 'SQ', 'SBAR'
parataxis_clause_tags = ['S', 'SBARQ', 'SINV', 'SQ', 'SBAR']
clause_re = re.compile('(' + '/|'.join(clause_tags) + ')')
pronoun_tags = ['PRP', 'PRPS']

aoa_df = pd.read_csv('reference/aoa/aoa_lemmas.csv')
with open('reference/word_frequency/subtlexus_lower.json', encoding='utf-8') as f:
    wf_dict = json.load(f)

def benepar_analysis(sent):
    features_lst = ['num_clauses', 'num_sbar', 'num_unk', 'depth_sum', 'clause_depth_sum', 'pronoun_sum', 'num_leaf_nps', 'num_nps', 
        'np_leaf_sum', 'clause_length_sum', 'paratactic_sum', 'root_parataxis_strict', 
        'root_parataxis_loose']
    features = Counter({x : 0 for x in features_lst}) # initialize to 0
    max_features = {
        'max_clause_depth' : 0,
        'max_depth' : 0, 
    }

    # global num_clauses, num_sbar, num_unk, depth_sum, max_depth, max_clause_depth, clause_depth_sum
    # global pronoun_sum, num_leaf_nps, num_nps, np_leaf_sum, clause_length_sum
    # global paratactic_sum, root_parataxis_strict, root_parataxis_loose
    paratactic_sum_local = 0

    xml = sexp_to_xml(sent._.parse_string)
    root = etree.fromstring(xml) 
    tree = etree.ElementTree(root)

    features['root_parataxis_loose'] = max(1, sum(int(bool(c.tag in parataxis_clause_tags)) 
                                                                    for c in root.getchildren()))
    features['root_parataxis_strict'] = find_parataxis_strict(root)
    features['num_sbar'] = sum(int(e.tag == 'SBAR') for e in root.iter())
    features['pronoun_sum'] = sum(int(e.tag in pronoun_tags) for e in root.iter())
    features['num_unk'] = sum(int(e.tag == 'UNK') for e in root.iter())
    
    for e in root.iter():
        tag = e.tag
        if tag in clause_tags: 
            features['num_clauses'] += 1
            features['clause_length_sum'] += sum(int(not(d.tag.startswith('PUNCT-') and bool(d.text))) for d in e.iterdescendants())
            
            paratactic_sum_local += sum(int(bool(c.tag in parataxis_clause_tags)) for c in e.getchildren())
        elif e.tag == 'NP':
            features['num_nps'] += 1
            is_leaf_np = True
            for c in e.iterdescendants():
                if c.text: 
                    if not(c.tag.startswith('PUNCT-') or c.tag == 'DT'): # ignore determiners and punctuation
                        features['np_leaf_sum'] += 1
                else:
                    is_leaf_np = False
            if is_leaf_np:
                features['num_leaf_nps'] += 1
        
        if e.text:
            path = tree.getpath(e)

            depth = len(re.findall('/', path))
            features['depth_sum'] += depth # Number of times '/' appears, excluding first
            max_features['max_depth'] = max(max_features['max_depth'], depth)

            clause_depth = len(clause_re.findall(path))
            features['clause_depth_sum'] += clause_depth
            max_features['max_clause_depth'] = max(max_features['max_clause_depth'], clause_depth)
            
    features['paratactic_sum'] = max(1, paratactic_sum_local)

    return features, max_features

def spacy_analysis(sent, uniq_words: set):
    features_lst = ['dep_dist_sum', 'num_words', 'words_before_root_sum', 'uniq_words', 
        'num_words_no_nums', 'num_stop_words', 'aoa_sum', 'aoa_count', 
        'aoa_stopless_sum', 'aoa_stopless_count', 'aoa_uniq_sum', 'aoa_uniq_count',
        'aoa_stopless_uniq_sum', 'aoa_stopless_uniq_count', 'wf_sum', 'wf_count', 
        'wf_stopless_sum', 'wf_stopless_count', 'wf_uniq_sum', 'wf_uniq_count',
        'wf_stopless_uniq_sum', 'wf_stopless_uniq_count']
    features = Counter({x : 0 for x in features_lst})
    max_features = {
        'max_dep_dist': 0, 
    }

    # global dep_dist_sum, num_words, words_before_root_sum, uniq_words, num_words_no_nums, num_stop_words
    # global max_dep_dist
    # global aoa_list, aoa_stopless_list, aoa_uniq_list, aoa_stopless_uniq_list
    # global wf_list, wf_stopless_list, wf_uniq_list, wf_stopless_uniq_list

    features['num_stop_words'] = sum(int(token.is_stop) for token in sent)

    for token in sent:
        if not(token.is_punct or token.is_space):
            features['num_words'] += 1
            dep_dist = abs(token.head.i - token.i)
            features['dep_dist_sum'] += dep_dist
            max_features['max_dep_dist'] = max(dep_dist, max_features['max_dep_dist'])
            
            if (wf := wf_dict.get(token.lower_)) is not None:
                features['wf_sum'] += wf
                features['wf_count'] += 1
                # wf_list.append(wf)
                if not token.is_stop:
                    features['wf_stopless_sum'] += wf
                    features['wf_stopless_count'] += 1
                    # wf_stopless_list.append(wf)

            if (aoa := aoa_of(token.lower_)) != -1:
                features['aoa_sum'] += aoa
                features['aoa_count'] += 1
                # aoa_list.append(aoa)
                if not token.is_stop:
                    features['aoa_stopless_sum'] += aoa
                    features['aoa_stopless_count'] += 1
                    # aoa_stopless_list.append(aoa)

            if token.i < sent.root.i: 
                features['words_before_root_sum'] += 1

            if not(token.like_num):
                if not(token.lower_ in uniq_words):
                    uniq_words.add(token.lower_)

                    if wf is not None:
                        features['wf_uniq_sum'] += wf
                        features['wf_uniq_count'] += 1
                        # wf_uniq_list.append(wf)
                        if not token.is_stop:
                            features['wf_stopless_uniq_sum'] += wf
                            features['wf_stopless_uniq_count'] += 1
                            # wf_stopless_uniq_list.append(wf)
                    
                    if aoa != -1:
                        features['aoa_uniq_sum'] += aoa
                        features['aoa_uniq_count'] += 1
                        # aoa_uniq_list.append(aoa)
                        if not token.is_stop:
                            features['aoa_stopless_uniq_sum'] += aoa
                            features['aoa_stopless_uniq_count'] += 1
                            # aoa_stopless_uniq_list.append(aoa)

                features['num_words_no_nums'] += 1
    return features, max_features, uniq_words
    
def ted_analysis(sent1, sent2):
    global ted_sum

    tree1 = Tree.from_text(apted_format(sent1._.parse_string))
    tree2 = Tree.from_text(apted_format(sent2._.parse_string))

    apted = APTED(tree1, tree2, )
    ted = apted.compute_edit_distance()
    ted_sum += ted

(NP (DT this) (SBAR (IN that) (S (NP (PRP he)) (VP (MD will) (VP (VB go))))) (: :) (SBAR (IN that) (S (NP (PRP i)) (VP (VBP am) (ADJP (JJ clear))))))
(NP (NP (DT this)) (SBAR (WHNP (WDT that)) (S (NP (PRP he)) (VP (MD will) (VP (VB go)))))) (: :) (SBAR (IN that) (S (NP (PRP i)) (VP (VBP am) (ADJP (JJ clear)))))

Main

In [64]:
results = pd.DataFrame([])

# TED Mode
#  - 'combinations': Averages the TED of every pair of sentences in a doc
#  - 'adjacent': Averages the TED of a sentence and the next sentence in a doc
ted_mode = 'adjacent'
# AoA Mode: If word is not in AoA list and multiple lemma matches exist
#  - 'max': Chooses max of lemma matches
#  - 'avg': Averages lemma matches
aoa_mode = 'max'
log = ""

for i, file in enumerate(os.scandir('data/text_jsons/')):
    if i == 2: break
    
    file_time = time.perf_counter()
    file_name = re.sub('\.json$', '', file.name)
    with open(file, encoding='utf-8') as f:
        metadata = json.load(f)
    text = metadata['text']
    text = text.replace('\n', ' ').strip()
    text = re.sub('\s{2,}', ' ', text)
    
    try:
        doc = nlp(text)
    except ValueError:
        error_text = f"ValueError in '{file_name}'. Likely exists too long sentence. Skipping."
        log += error_text + '\n'
        continue
    except:
        error_text = f"Some other error occured in '{file_name}'. Skipping."
        log += error_text + '\n'
        continue

    sents = list(doc.sents)

    # Doc-level
    num_tokens = len(doc)
    num_sents = len(sents)
    ted_sum = 0

    features_lst = ['num_clauses', 'num_sbar', 'num_unk', 'depth_sum', 'max_depth', 
        'max_clause_depth', 'clause_depth_sum', 'pronoun_sum', 'num_leaf_nps', 'num_nps', 
        'np_leaf_sum', 'clause_length_sum', 'paratactic_sum', 'root_parataxis_strict', 
        'root_parataxis_loose', 'dep_dist_sum', 'num_words', 'words_before_root_sum', 'uniq_words', 
        'num_words_no_nums', 'num_stop_words', 'max_dep_dist', 'aoa_sum', 'aoa_count', 
        'aoa_stopless_sum', 'aoa_stopless_count', 'aoa_uniq_sum', 'aoa_uniq_count',
        'aoa_stopless_uniq_sum', 'aoa_stopless_uniq_count', 'wf_sum', 'wf_count', 
        'wf_stopless_sum', 'wf_stopless_count', 'wf_uniq_sum', 'wf_uniq_count',
        'wf_stopless_uniq_sum', 'wf_stopless_uniq_count']
    features = Counter({x : 0 for x in features_lst})
    uniq_words = set()

    for sent in sents:
        benepar_time = time.perf_counter()
        b_features, b_max_features = benepar_analysis(sent)
        features.update(b_features)
        for feature in b_max_features:
            features[feature] = max(features.get(feature), b_max_features[feature])
        benepar_time = time.perf_counter() - benepar_time

        spacy_time = time.perf_counter()
        s_features, s_max_features, sent_vocab = spacy_analysis(sent, uniq_words)
        features.update(s_features)
        for feature in s_max_features:
            features[feature] = max(features.get(feature), s_max_features[feature])
        uniq_words.update(sent_vocab)
        spacy_time = time.perf_counter() - spacy_time

    # TREE EDIT DISTANCE
    ted_time = time.perf_counter()
    if ted_mode == 'adjacent':    
        for i in range(num_sents - 1):
            ted_analysis(sents[i], sents[i + 1])
        ted_avg = ted_sum / (num_sents - 1)
    elif ted_mode == 'combinations':
        for sent1, sent2 in combinations(sents, 2):
            ted_analysis(sent1, sent2)
        ted_avg = ted_sum / comb(num_sents, 2)
    else:
        print('Invalid ted_mode:', ted_mode)
        ted_avg = -1
    ted_time = time.perf_counter() - ted_time

    summary = {
        # File-level
        'date' : metadata['date'],
        'pres_name' : metadata['pres_name'],
        'byline' : metadata['byline'],
        'title' : metadata['title'],

        # Performance time
        'benepar_analysis_time' : benepar_time,
        'spacy_analysis_time' : spacy_time,
        'tree_edit_distance_time' : ted_time,
        'total_file_analysis_time' : time.perf_counter() - file_time,

        # Doc-level
        'num_tokens' : num_tokens,
        'num_sentences' : num_sents, 
        # 'avg_ted_adj' : ted_avg_adj,
        # 'avg_ted_comb' : ted_avg_comb,
        f'avg_tree_edit_dist_{ted_mode}' : ted_avg,

        # Benepar
        'avg_node_depth' : features['depth_sum'] / num_tokens, 
        'max_node_depth' : features['max_depth'], # Equivalent to tree height
        'avg_node_clause_depth' : features['clause_depth_sum'] / num_tokens,
        'max_node_clause_depth' : features['max_clause_depth'],
        'avg_clause_length' : features['clause_length_sum'] / features['num_clauses'],
        'clauses_per_sent' : features['num_clauses'] / num_sents, 
        'sbars_per_sent' : features['num_sbar'] / num_sents,
        'pronouns_per_sent' : features['pronoun_sum'] / num_sents,
        'pronouns_per_clause' : features['pronoun_sum'] / features['num_clauses'],
        'pronoun_prop_of_leaf_nps' : features['pronoun_sum'] / features['num_leaf_nps'],
        'avg_num_np_modifiers' : features['np_leaf_sum'] / features['num_nps'],
        'loose_parataxis_per_sent' : features['paratactic_sum'] / num_sents,
        'root_parataxis_per_sent_strict' : features['root_parataxis_strict'] / num_sents,
        'root_parataxis_per_sent_loose' : features['root_parataxis_loose'] / num_sents,
        'num_unk' : features['num_unk'],

        # spaCy
        'num_words' : features['num_words'],
        'avg_dependency_distance' : features['dep_dist_sum'] / features['num_words'],
        'max_dependency_distance' : features['max_dep_dist'],
        'avg_sentence_length_by_tok' : num_tokens / num_sents, 
        'avg_sentence_length_by_word' : features['num_words'] / num_sents,
        'avg_words_before_root' : features['words_before_root_sum'] / num_sents,
        'num_uniq_words' : len(uniq_words), 
        'proportion_uniq' : len(uniq_words) / features['num_words_no_nums'],
        'stop_words_per_clause' : features['num_stop_words'] / features['num_clauses'],
        'stop_words_per_sentence' : features['num_stop_words'] / num_sents,
        f'avg_aoa_{aoa_mode}' : features['aoa_sum'] / features['aoa_count'],
        f'avg_aoa_uniq_{aoa_mode}' : features['aoa_uniq_sum'] / features['aoa_uniq_count'],
        f'avg_stopless_aoa_{aoa_mode}' : features['aoa_stopless_sum'] / features['aoa_stopless_count'],
        f'avg_stopless_aoa_uniq_{aoa_mode}' : features['aoa_stopless_uniq_sum'] / features['aoa_stopless_uniq_count'],
        'avg_word_freq' : features['wf_sum'] / features['wf_count'],
        'avg_word_freq_uniq' : features['wf_uniq_sum'] / features['wf_uniq_count'],
        'avg_word_freq_stopless' : features['wf_stopless_sum'] / features['wf_stopless_count'],
        'avg_word_freq_stopless_uniq' : features['wf_stopless_uniq_sum'] / features['wf_stopless_uniq_count'],
    }      
    
    results[file_name] = summary
    
print("Error Log:")
print(log)
pd.set_option('display.precision', 2)
print(results)
results.to_csv(f"results/{datetime.now().strftime('%m-%d-%Y_%H-%M')}.csv")


C:\Users\cedch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\distributions\distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[<Element NP at 0x2b1da999640>, <Element PUNCT-COLON at 0x2b1da9a7000>, <Element S at 0x2b1c4535400>, <Element PUNCT-PERIOD at 0x2b1d5aa7b40>]
['NP', 'S']
[<Element PP at 0x2b1d48a7080>, <Element PUNCT-COMMA at 0x2b1da1692c0>, <Element NP at 0x2b1d5a71f80>, <Element VP at 0x2b1c4534d80>, <Element PUNCT-PERIOD at 0x2b1de1df5c0>]
['PP', 'NP', 'VP']
[<Element PP at 0x2b20d58e880>, <Element PUNCT-COMMA at 0x2b20d58f9c0>, <Element NP at 0x2b20d58de00>, <Element VP at 0x2b20d58d000>, <Element PUNCT-PERIOD at 0x2b20d58c040>]
['PP', 'NP', 'VP']
[<Element PP at 0x2b20d58d740>, <Element NP at 0x2b1da2d8500>, <Element VP at 0x2b1b5dc84c0>, <Element PUNCT-PERIOD at 0x2b1b5dcb440>]
['PP', 'NP', 'VP']
[<Element NP at 0x2b1dead4980>, <Element VP at 0x2b1dead5740>, <Element PUNCT-PERIOD at 0x2b1dead78c0>]
['NP', 'VP']
[<Element PP at 0x2b1dc116b80>, <Element PUNCT-COMMA at 0x2b2160f1100>, <Element NP at 0x2b1de0e14c0>, <Element VP at 0x2b1de0e06c0>, <Element PUNCT-PERIOD at 0x2b1de0e1d00>]
['PP', 'NP'

C:\Users\cedch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\distributions\distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[<Element NP at 0x2b1da1da1c0>, <Element PUNCT-COLON at 0x2b1d5acab00>, <Element S at 0x2b1d5aa41c0>, <Element PUNCT-PERIOD at 0x2b1de9e2dc0>]
['NP', 'S']
[<Element SBAR at 0x2b1b5de2cc0>, <Element PUNCT-COMMA at 0x2b1b5de1200>, <Element NP at 0x2b1de2dfc80>, <Element VP at 0x2b1daba4c00>, <Element PUNCT-PERIOD at 0x2b1daba4e00>]
['SBAR', 'NP', 'VP']
[<Element PP at 0x2b1da972040>, <Element NP at 0x2b1da972300>, <Element VP at 0x2b1da971d40>, <Element PUNCT-PERIOD at 0x2b1da972f40>]
['PP', 'NP', 'VP']
[<Element NP at 0x2b1dc15c3c0>, <Element PUNCT-COLON at 0x2b1d5aa5dc0>, <Element SBAR at 0x2b1d5aa6780>, <Element PUNCT-PERIOD at 0x2b1d5aa7080>]
['NP', 'SBAR']
Error Log:

                                              1789_Washington_Inaugural_Address  \
date                                                                 1789-04-30   
pres_name                                                     George Washington   
byline                          1st President of the United States: 178

PCA

In [170]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.decomposition import FactorAnalysis

# print(results['num_tokens':])
np.set_printoptions(precision=100)

results_data = results['num_tokens':].drop(['num_tokens', 'num_words', 
    'num_sentences', 'num_unk', 'num_uniq_words', 'avg_word_freq_uniq'])
# results_data = results['num_tokens':]
results_std = StandardScaler().fit_transform(results_data).T
# results_std = normalize(results_data).T

# print([np.max(i)-np.min(i) for i in results_std])

# print(results_std.shape)
# fa = FactorAnalysis(n_components=3)
# f = fa.fit(results_std).components_.T
# # print(np.sum(f.T[0]))
# # print(f)

# # print(fa.fit_transform(results_std.T))

# faf = pd.DataFrame(f, index=list(results_data.index), columns=['1', '2', '3'])
# print(faf.nlargest(10, ['1']))



# PCA
pca = PCA(n_components=3).fit(results_std)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
# print(pca.components_.shape)
# print(pca.components_.T)

comps = pd.DataFrame(pca.components_.T, index=list(results_data.index), columns=['1', '2', '3'])
m = comps.abs().nlargest(20, ['1']).index
print(comps.loc[m])

print('num_tokens' in comps.index)

# print('-'*100)

# cov = np.cov(results_std.T)

# evals, evecs = np.linalg.eig(cov)

# print(evecs)
# print(evecs.shape)

# explained_variances = []
# for i in range(len(evals)):
#     explained_variances.append(evals[i] / np.sum(evals))
 
# print(np.sum(explained_variances), '\n', explained_variances)



[0.798829799226471   0.13868336305891654 0.03236415372895239]
0.96987731601434
                                1         2         3
avg_word_freq_stopless       0.84  1.72e-01 -4.53e-01
avg_word_freq_stopless_uniq  0.43  4.56e-02  8.78e-01
avg_tree_edit_dist_adjacent -0.18  2.06e-01  6.46e-02
max_dependency_distance     -0.17  8.83e-01 -2.74e-02
avg_sentence_length_by_tok  -0.10  1.34e-02 -1.67e-03
avg_sentence_length_by_word -0.10  3.90e-03 -2.90e-03
avg_clause_length           -0.08  7.60e-03 -2.08e-02
stop_words_per_sentence     -0.06 -2.67e-02 -4.49e-03
max_node_depth              -0.06  1.76e-01  6.01e-02
avg_words_before_root       -0.05 -3.63e-02 -1.58e-02
avg_node_depth              -0.03 -5.57e-02 -1.47e-02
stop_words_per_clause       -0.03 -7.92e-02 -2.89e-02
avg_num_np_modifiers        -0.03 -6.82e-02 -1.99e-02
clauses_per_sent            -0.03 -7.66e-02 -2.25e-02
avg_word_freq               -0.03 -2.72e-02  2.35e-03
avg_stopless_aoa_max        -0.03 -8.39e-02 -2.90e-02
pro

- vocab measures
    - measures of polysemy
- weight ted differently --> should adding/removing be weighted less because it simply indicates a different length sentence?
    - how to remove determiners?
- make scraper more robust
- unit testing for parataxis measures

- word not in AoA list 
    - opt 1: average lemmas
    - opt 2: take max

- cosine similarity
    - Something at the end???

- PCA and RFE
- maybe paratactic should go back to the regular clause tags because we don't want S:SBAR cuz they're related


Questions
- Workaround for the spacy max length? <code>ValueError: Sentence of length 965 (in sub-word tokens) exceeds the maximum supported length of 512</code>
- How to compare syntax trees better?
    - Should I do TED weighting?
- Visually displaying / analyzing the data
- Code not running on harris...

- View: https://scholarworks.gsu.edu/cgi/viewcontent.cgi?article=1035&context=alesl_diss
- View: http://cohmetrix.memphis.edu/cohmetrixhome/documentation_indices.html#Complexity
- MORE RESEARCH NEEDED: compare tree similarity (SEARCH TREE EDIT DISTANCE) of sentences in doc, pq-gram distance
(CITING APTED: https://pypi.org/project/apted/#description)

In [42]:
p_sum = 0
p_suml = 0
d = nlp("This oath I am now about to take, and in your presence: That if it shall be found during my administration of the Government I have in any instance violated willingly or knowingly the injunctions thereof, I may (besides incurring constitutional punishment) be subject to the upbraidings of all who are now witnesses of the present solemn ceremony.")
d = nlp("this that he will go: that i am clear")
# d=nlp("to go: that i am clear")
# returns p_sum 2, p_suml 4
s = list(d.sents)[0]
print(s)
xml = sexp_to_xml(s._.parse_string)
print(s._.parse_string)
print(xml)
root = etree.fromstring(xml) 
tree = etree.ElementTree(root)
print(root.getchildren())
print(root)

p_sum += find_parataxis_strict(root)
for e in root.iter():
    tag = e.tag
    if tag in clause_tags:
        p_suml += sum(int(bool(c.tag in parataxis_clause_tags)) for c in e.getchildren())


print(p_sum, p_suml)

this that he will go: that i am clear
(FRAG (NP (NP (DT this)) (SBAR (WHNP (WDT that)) (S (NP (PRP he)) (VP (MD will) (VP (VB go)))))) (: :) (SBAR (IN that) (S (NP (PRP i)) (VP (VBP am) (ADJP (JJ clear))))))
<FRAG><NP><NP><DT>this</DT></NP><SBAR><WHNP><WDT>that</WDT></WHNP><S><NP><PRP>he</PRP></NP><VP><MD>will</MD><VP><VB>go</VB></VP></VP></S></SBAR></NP><PUNCT-COLON>:</PUNCT-COLON><SBAR><IN>that</IN><S><NP><PRP>i</PRP></NP><VP><VBP>am</VBP><ADJP><JJ>clear</JJ></ADJP></VP></S></SBAR></FRAG>
[<Element NP at 0x2b1daae16c0>, <Element PUNCT-COLON at 0x2b1da7e5000>, <Element SBAR at 0x2b1da7e4780>]
<Element FRAG at 0x2b1da7ab300>
[<Element NP at 0x2b1c45ab200>, <Element PUNCT-COLON at 0x2b1da7e47c0>, <Element SBAR at 0x2b1da7e5000>]
['NP', 'SBAR']
1 0


Download Libraries

In [ ]:
! pip install benepar
! pip install spacy
! pip install apted
! pip install bs4
! python -m spacy download en_core_web_md

In [49]:
a=[("13.5",100)]
b=[("14.5",100), ("15.5", 100)]
c=[("15.5",100), ("16.5", 100)]
b={"14.5":100, "15.5": 100}
c={"15.5":100, "16.5": 100}
input=[b, c]

from collections import Counter
print(Counter(c))
print(sum(
    (Counter(x) for x in input),
    Counter()))

Counter({'15.5': 100, '16.5': 100})
Counter({'15.5': 200, '14.5': 100, '16.5': 100})
